In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00


In [12]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def preprocess_text(text, tokenizer):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    return inputs

def train_model(model, train_inputs, train_labels, num_epochs=3, batch_size=16):
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    model.to(device)

    train_dataset = torch.utils.data.TensorDataset(train_inputs['input_ids'],
                                                  train_inputs['attention_mask'],
                                                  train_labels)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {total_loss:.4f}')

    # Save the trained model
    model.save_pretrained("saved_model")

def classify_sentence(model, sentence, tokenizer):
    model.eval()
    input_data = preprocess_text(sentence, tokenizer)
    input_ids = input_data['input_ids'].to(device)
    attention_mask = input_data['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_label = torch.argmax(logits, dim=1).item()

    return predicted_label

if __name__ == '__main__':
    # Load pre-trained BERT model and tokenizer
    model_name = 'bert-base-uncased'
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

    # Example training data (paragraphs)
    train_paragraphs = [
        "Yoga is an ancient practice that originated in India. It combines physical postures, breathing exercises, and meditation. Many people practice yoga for its numerous benefits, including improved flexibility, stress reduction, and mental well-being.",
        "Basketball is a popular sport played around the world. It involves two teams competing to score points by shooting a ball into the opposing team's basket. While basketball requires physical fitness and coordination, it is not directly related to yoga."
    ]
    train_labels = [1, 0]  # 1 for yoga-related, 0 for not yoga-related

    # Example test sentence
    test_sentence = "I enjoy practicing yoga and feel more relaxed afterwards."

    # Preprocess and train the model
    train_inputs = preprocess_text(train_paragraphs, tokenizer)
    train_labels = torch.tensor(train_labels)
    train_model(model, train_inputs, train_labels)

    # Save the trained model and tokenizer
    model.save_pretrained("saved_model")
    tokenizer.save_pretrained("saved_tokenizer")

    # Load the saved model and tokenizer for future training
    loaded_model = BertForSequenceClassification.from_pretrained("saved_model")
    loaded_tokenizer = BertTokenizer.from_pretrained("saved_tokenizer")

    # Perform inference on the test sentence using the loaded model and tokenizer
    predicted_label


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Epoch 1/3 - Loss: 0.5409
Epoch 2/3 - Loss: 0.6999
Epoch 3/3 - Loss: 0.6676
